In [2]:
import pandas as pd
from datasets import Dataset, load_from_disk
from tqdm import tqdm

def extract_templates_set(dataset, dataset_name="Dataset"):
    """
    데이터셋에서 prompt_text의 분자 정보를 제거하고 유니크한 템플릿 집합(Set)을 반환합니다.
    """
    # 1. 제거할 그래프 토큰 정의
    graph_tokens = "<GRAPH>" + "<mol>" * 32 + "</GRAPH>"
    
    unique_templates = set()
    
    print(f"Extracting templates from {dataset_name}...")
    for data in tqdm(dataset):
        prompt_text = data['prompt_text']
        input_mol_string = data['input_mol_string']
        
        # 2. 제거할 타겟 문자열 (SELFIES + GRAPH)
        target_to_remove = input_mol_string + graph_tokens
        
        # 3. 문자열 치환
        clean_template = prompt_text.replace(target_to_remove, r"{Mol Input String + graph feature")
        
        # (선택) 앞뒤 공백 제거 (replace 후 남은 공백 정리)
        clean_template = clean_template.strip()
        
        unique_templates.add(clean_template)
        
    return unique_templates

def compare_datasets(dataset1, dataset2, name1="Dataset A", name2="Dataset B"):
    # 1. 각 데이터셋에서 템플릿 Set 추출
    set1 = extract_templates_set(dataset1, name1)
    set2 = extract_templates_set(dataset2, name2)
    
    # 2. 집합 연산 수행
    common = set1.intersection(set2)  # 교집합 (둘 다 있음)
    only_in_1 = set1.difference(set2) # 1에만 있음
    only_in_2 = set2.difference(set1) # 2에만 있음
    
    # 3. 결과 리포트 출력
    print("\n" + "="*60)
    print(f"Comparison Result: {name1} vs {name2}")
    print("="*60)
    print(f"Total Unique Templates in {name1}: {len(set1)}")
    print(f"Total Unique Templates in {name2}: {len(set2)}")
    print("-" * 60)
    print(f"∩ Common Templates (Both have): {len(common)}")
    print(f"- Unique to {name1} (Only in A): {len(only_in_1)}")
    print(f"- Unique to {name2} (Only in B): {len(only_in_2)}")
    print("="*60)
    
    # 4. 상세 예시 출력 (옵션)
    if only_in_1:
        print(f"\n[Example] Templates found ONLY in {name1} (First 3):")
        for i, t in enumerate(list(only_in_1)[:3]):
            print(f"{i+1}. {t[:100]}...") # 너무 길면 자름
            
    if only_in_2:
        print(f"\n[Example] Templates found ONLY in {name2} (First 3):")
        for i, t in enumerate(list(only_in_2)[:3]):
            print(f"{i+1}. {t[:100]}...")

    return set1, set2, common, only_in_1, only_in_2

# --- 실행 예시 (가상 데이터) ---

# 데이터셋 A (기존 템플릿 포함)
data_A = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download_InstructGraph_bace")
data_B = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writier_bace")
ds_A = Dataset.from_list(data_A)
ds_B = Dataset.from_list(data_B)

# 비교 함수 실행
set_A, set_B, common, only_A, only_B = compare_datasets(ds_A, ds_B, "Old_Dataset", "New_Dataset")

Extracting templates from Old_Dataset...


100%|██████████| 152/152 [00:00<00:00, 1747.50it/s]


Extracting templates from New_Dataset...


100%|██████████| 152/152 [00:00<00:00, 1786.37it/s]


Comparison Result: Old_Dataset vs New_Dataset
Total Unique Templates in Old_Dataset: 13
Total Unique Templates in New_Dataset: 13
------------------------------------------------------------
∩ Common Templates (Both have): 13
- Unique to Old_Dataset (Only in A): 0
- Unique to New_Dataset (Only in B): 0


In [3]:
sorted(set_A)

['<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. \n\nA molecule {Mol Input String + graph feature is given; what could be the biological activity against BACE-1? [/INST]',
 '<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. \n\nBased on the given molecule: {Mol Input String + graph feature, what biological activity could potentially be observed against BACE-1? [/INST]',
 '<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. \n\nCan you tell me the biological act

In [4]:
sorted(set_B)

['<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. \n\nA molecule {Mol Input String + graph feature is given; what could be the biological activity against BACE-1? [/INST]',
 '<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. \n\nBased on the given molecule: {Mol Input String + graph feature, what biological activity could potentially be observed against BACE-1? [/INST]',
 '<s>[INST] You are a helpful assistant for molecular chemistry, to address tasks including molecular property classification, molecular property regression, chemical reaction prediction, molecule captioning, molecule generation. \n\nCan you tell me the biological act